In [ ]:
!pip install -q transformers accelerate safetensors einops

In [ ]:
!pip install -q transformers accelerate safetensors optimum einops flash-attn bitsandbytes

In [ ]:
import torch


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install -q hqq transformers[torch] optimum

In [ ]:
# Cell to Add: Install vLLM
!pip install vllm -q
# Note: Depending on the Colab environment's CUDA/Driver version, you might need a specific vLLM build.
# If the above fails, check vLLM documentation for compatibility.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 

In [ ]:
# Cell 1: Install necessary libraries
# We primarily need vLLM and transformers (for the tokenizer)
!pip install vllm==0.5.1  # Specify a version for potentially better stability
!pip install transformers==4.42.4 -q # Keep transformers version consistent
print("--- Installation Complete ---")

  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.0-py3-none-any.whl (10.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
--- Installation Complete ---


In [ ]:
!pip install vllm --upgrade

  Using cached vllm-0.8.3-cp38-abi3-manylinux1_x86_64.whl.metadata (27 kB)
  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
  Using cached lm_format_enforcer-0.10.11-py3-none-any.whl.metadata (17 kB)
  Using cached xformers-0.0.29.post2-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using 

In [ ]:
# Cell 2: Imports and Configuration Setup
import os
import time
import torch
import gc
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional

# VLLM Imports
from vllm import LLM, SamplingParams

# Keep tokenizer import from transformers
from transformers import AutoTokenizer

# Set device (vLLM typically manages device placement, but good practice)
device_name = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device_name}")

# Check GPU info (still useful)
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    print(f"GPU: {gpu_name}")
    print(f"Memory: {gpu_memory:.2f} GB")
else:
    print("No GPU detected, using CPU")

# --- Configuration Dataclasses ---
@dataclass
class BenchmarkConfig:
    """Configuration for the benchmark run."""
    model_id: str
    max_input_length: int = 128
    max_output_length: int = 128
    num_benchmark_prompts: int = 32 # Target concurrency
    tensor_parallel_size: int = 1 # For single T4 GPU
    gpu_memory_utilization: float = 0.90 # vLLM: Use 90% of GPU memory for KV cache
    quantization: Optional[str] = None # e.g., 'awq', 'gptq'. vLLM might detect automatically.
        # --- ADD THESE TWO LINES ---
    max_model_len_override: Optional[int] = None # Optional: Override model's max length
    max_num_batched_tokens_override: Optional[int] = None # Optional: Override vLLM batch token limit


    # LoRA arguments for vLLM
    enable_lora: bool = False
    lora_modules: Optional[List[Dict[str, Any]]] = field(default_factory=list) # Structure: [{"lora_name": "...", "lora_int_id": 1, "lora_local_path": "..."}]
    max_loras: int = 1
    max_lora_rank: int = 16 # Common default, adjust if needed for specific adapters

@dataclass
class RequestOutput:
    """Simple struct to hold results for each request."""
    id: int
    prompt: str
    result: str = ""
    metrics: Dict[str, Any] = field(default_factory=dict)
    start_time: float = None # Note: In vLLM batch, start/end might be approximate
    end_time: float = None

print("--- Imports and Config Setup Complete ---")

Using device: cuda
GPU: Tesla T4
Memory: 14.74 GB
--- Imports and Config Setup Complete ---


In [ ]:
# Cell 3: Benchmark Function using vLLM
def run_vllm_benchmark(tokenizer: AutoTokenizer, config: BenchmarkConfig, llm: LLM):
    """
    Run a benchmark using the initialized vLLM engine.
    """
    print("\n=== Running vLLM Benchmark ===")
    print(f"Model: {config.model_id}")
    print(f"Target Concurrency: {config.num_benchmark_prompts}")
    print(f"Input/Output Length: {config.max_input_length}/{config.max_output_length}")

    # --- Prepare Benchmark Prompts ---
    # Create a prompt designed to be exactly config.max_input_length tokens long
    base_prompt = "Explain the differences between transformer models like BERT, GPT, and T5. Include details about their architecture, training objectives, and typical use cases."
    padding_text = " This analysis should cover model size, parameter count, and computational requirements."

    # Encode, truncate/pad to exact length, then decode back to ensure exact token count
    encoded = tokenizer.encode(base_prompt)
    if len(encoded) < config.max_input_length:
         # Extend prompt if too short (simple repetition)
        target_len = config.max_input_length
        while len(encoded) < target_len:
             encoded.extend(tokenizer.encode(padding_text, add_special_tokens=False))
             if len(encoded) > target_len + 20: # Avoid excessive padding text
                  break # Safety break
    # Truncate if too long
    if len(encoded) > config.max_input_length:
        encoded = encoded[:config.max_input_length]

    # Decode the precisely sized token list back into a string prompt
    # Ensure skip_special_tokens=True if the tokenizer adds BOS/EOS by default in encode
    final_prompt_string = tokenizer.decode(encoded, skip_special_tokens=True)
    final_length = len(tokenizer.encode(final_prompt_string)) # Verify final length

    # Handle potential small discrepancies due to decoding/re-encoding edge cases
    if final_length != config.max_input_length:
         print(f"Warning: Final prompt length ({final_length}) differs slightly from target ({config.max_input_length}). Adjusting...")
         # Simple truncation as last resort
         encoded = tokenizer.encode(final_prompt_string)[:config.max_input_length]
         final_prompt_string = tokenizer.decode(encoded, skip_special_tokens=True)
         final_length = len(tokenizer.encode(final_prompt_string))

    print(f"Using benchmark prompt with token length: {final_length}")
    prompts = [final_prompt_string] * config.num_benchmark_prompts
    # It's often faster to give vLLM token IDs directly
    prompt_token_ids = [encoded] * config.num_benchmark_prompts # Use the pre-calculated precise token list

    # --- Configure vLLM Sampling Parameters ---
    sampling_params = SamplingParams(
        n=1, # Number of output sequences per prompt
        temperature=0.0, # Deterministic for benchmark consistency
        top_p=1.0,
        max_tokens=config.max_output_length,
        # ignore_eos=True, # Set True if you MUST generate exactly max_output_length tokens
                           # Set False (default) allows stopping early on EOS token. Benchmark might vary slightly.
        skip_special_tokens=True # Usually desired
    )

    # --- Prepare LoRA Request if enabled ---
    lora_request = None
    if config.enable_lora and config.lora_modules:
        # Assuming only one LoRA for the benchmark run
        lora_info = config.lora_modules[0]
        lora_request = LoRARequest(
            lora_name=lora_info["lora_name"],
            lora_int_id=lora_info["lora_int_id"]
        )
        print(f"Using LoRA request: {lora_request}")


    # --- Run Inference ---
    print(f"\nSubmitting {len(prompts)} prompts to vLLM engine...")
    torch.cuda.synchronize() # Ensure previous work is done
    start_time = time.time()

    # vLLM's core generate call - handles batching and concurrency internally
    vllm_outputs = llm.generate(
        prompts=None, # Use prompt_token_ids for potential speedup
        sampling_params=sampling_params,
        prompt_token_ids=prompt_token_ids,
        lora_request=lora_request # Pass LoRA request if enabled
        )

    torch.cuda.synchronize() # Ensure generation is complete
    end_time = time.time()
    total_time = end_time - start_time
    print(f"vLLM generation finished in {total_time:.2f} seconds.")

    # --- Process Results and Calculate Metrics ---
    results = []
    total_input_tokens_processed = 0
    total_output_tokens_processed = 0

    for i, output in enumerate(vllm_outputs):
        # vLLM output object contains prompt, generated text, token ids, etc.
        req_output = RequestOutput(id=i, prompt=output.prompt) # Original prompt string
        req_output.start_time = start_time # Use overall batch start time
        req_output.end_time = end_time # Use overall batch end time
        req_output.result = output.outputs[0].text # The generated text

        input_len = len(output.prompt_token_ids)
        output_len = len(output.outputs[0].token_ids)
        total_input_tokens_processed += input_len
        total_output_tokens_processed += output_len

        # Store individual metrics if needed, though aggregate is primary
        req_output.metrics = {
            "input_tokens": input_len,
            "output_tokens": output_len,
            "total_tokens": input_len + output_len,
        }
        results.append(req_output)

    # Calculate aggregate metrics using the dedicated function
    metrics = calculate_vllm_aggregate_metrics(
        results, start_time, end_time, total_input_tokens_processed, total_output_tokens_processed
    )

    # --- Print Benchmark Summary ---
    print("\n--- vLLM Benchmark Results ---")
    print(f"Total requests submitted: {config.num_benchmark_prompts}")
    print(f"Successful requests processed: {metrics.get('successful_requests', 0)}")
    print(f"Total input tokens processed: {metrics.get('total_input_tokens', 0)}")
    print(f"Total output tokens processed: {metrics.get('total_output_tokens', 0)}")
    print(f"Total tokens (in+out): {metrics.get('total_tokens', 0)}")
    print(f"Total wall time: {metrics.get('total_time_seconds', 0):.2f} seconds")
    print(f"Aggregate throughput: {metrics.get('aggregate_throughput', 0):.2f} tokens/sec")
    print("---------------------------------")
    print(f"Target throughput: 200 tokens/sec")

    if metrics.get('aggregate_throughput', 0) >= 200:
        print("✅ BENCHMARK PASSED: Throughput meets or exceeds target!")
    else:
        print("❌ BENCHMARK FAILED: Throughput below target.")
        print("   Consider adjusting vLLM parameters like 'gpu_memory_utilization',")
        print("   'max_num_batched_tokens', or trying explicit quantization ('awq', 'gptq').")

    return metrics, results

In [ ]:
# Cell 4: Aggregate Metrics Calculation Function for vLLM
def calculate_vllm_aggregate_metrics(
    results: List[RequestOutput],
    start_time: float,
    end_time: float,
    total_input_tokens: int,
    total_output_tokens: int
    ):
    """
    Calculate aggregate metrics for a vLLM benchmark run.
    Uses overall start/end time and pre-summed tokens for accuracy.
    """
    total_requests_processed = len(results)
    total_time_seconds = end_time - start_time
    total_tokens_processed = total_input_tokens + total_output_tokens

    aggregate_throughput = total_tokens_processed / total_time_seconds if total_time_seconds > 0 else 0

    return {
        "total_input_tokens": total_input_tokens,
        "total_output_tokens": total_output_tokens,
        "total_tokens": total_tokens_processed,
        "total_time_seconds": total_time_seconds,
        "aggregate_throughput": aggregate_throughput,
        "successful_requests": total_requests_processed,
    }

In [ ]:
# Cell 5: Main Execution Function (FINAL - Returns Objects, No Interactive Loop)
def main():
    # --- Configuration for Maximum Throughput ---
    # MUST use a quantized model compatible with vLLM and T4 VRAM
    # AWQ is recommended for speed. Ensure this exact model ID exists and is compatible.
    default_model = "TheBloke/Mistral-7B-Instruct-v0.1-AWQ"
    detected_quantization = "awq" # Set based on the chosen model ID

    # Override vLLM defaults potentially beneficial for T4 and this task
    # Limit model length to slightly more than needed (128+128=256) to save KV cache memory
    max_len_override = 512
    # Max batched tokens: Let vLLM decide initially, adjust if OOM occurs during generation
    max_batched_tokens_override = None # Keep None for now

    config = BenchmarkConfig(
        model_id=default_model,
        enable_lora=False, # Explicitly disable LoRA
        quantization=detected_quantization, # Use the quantization type of the model
        max_model_len_override=max_len_override,
        max_num_batched_tokens_override=max_batched_tokens_override
        )

    print("--- Configuration for Max Throughput (Quantized) ---")
    print(f"Model ID: {config.model_id}")
    print(f"Quantization: {config.quantization}")
    print(f"LoRA Disabled")
    if config.max_model_len_override:
         print(f"Max Model Length Override: {config.max_model_len_override}")
    if config.max_num_batched_tokens_override:
         print(f"Max Batched Tokens Override: {config.max_num_batched_tokens_override}")
    print("----------------------------------------------------")

    # --- Load Tokenizer ---
    print(f"\nLoading tokenizer for {config.model_id}...")
    try:
        # AWQ models often need legacy=False
        tokenizer = AutoTokenizer.from_pretrained(config.model_id, trust_remote_code=True, legacy=False)
    except TypeError:
         print("Tokenizer does not support 'legacy' argument, loading without it.")
         tokenizer = AutoTokenizer.from_pretrained(config.model_id, trust_remote_code=True)


    # Add padding token if missing
    if tokenizer.pad_token is None:
        if tokenizer.eos_token is not None:
            print("Warning: No pad token found, using EOS token as pad token.")
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.pad_token_id = tokenizer.eos_token_id
        else:
             print("Warning: No pad or EOS token found. Adding a new pad token '[PAD]'.")
             tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.padding_side = 'left'
    print("Tokenizer loaded.")
    print(f"Padding side: {tokenizer.padding_side}")
    print(f"Pad token: '{tokenizer.pad_token}', ID: {tokenizer.pad_token_id}")

    # --- Initialize vLLM Engine ---
    print(f"\nInitializing vLLM engine with AWQ model...")
    # Prepare arguments
    llm_args = {
        "model": config.model_id,
        "tokenizer": config.model_id,
        "tensor_parallel_size": config.tensor_parallel_size,
        "gpu_memory_utilization": config.gpu_memory_utilization,
        "quantization": config.quantization, # Crucial: Pass 'awq'
        "trust_remote_code": True,
        "enable_lora": config.enable_lora,
        "max_loras": config.max_loras,
        "max_lora_rank": config.max_lora_rank,
        "dtype": 'half' # Explicitly set to half/float16 for AWQ & T4
    }
    if config.max_model_len_override:
        llm_args["max_model_len"] = config.max_model_len_override
    if config.max_num_batched_tokens_override:
        llm_args["max_num_batched_tokens"] = config.max_num_batched_tokens_override

    llm_engine = None
    try:
        start_init = time.time()
        llm_engine = LLM(**llm_args)
        end_init = time.time()
        print(f"vLLM Engine Initialized in {end_init - start_init:.2f} seconds.")
    except Exception as e:
        print(f"\n--- ERROR initializing vLLM Engine ---")
        print(e)
        print("Please check model ID, AWQ compatibility (vLLM needs specific dependencies for AWQ), and VRAM.")
        # Return None values if initialization fails
        return None, None, None

    # --- Run Benchmark ---
    # Note: benchmark function uses the passed tokenizer, config, llm_engine
    benchmark_metrics, benchmark_results = run_vllm_benchmark(tokenizer, config, llm_engine)

    # --- Main function is done, return objects needed for interactive mode ---
    print("\n--- Benchmark Complete ---")
    return llm_engine, tokenizer, config # Return engine, tokenizer, config

In [ ]:
# Cell 6: Script Entry Point (FINAL - Includes Interactive Loop)
if __name__ == "__main__":
    llm_engine = None # Initialize to None
    tokenizer = None
    config = None

    try:
        # Run main logic (init, benchmark) and get objects
        # This call might take a long time due to downloads and initialization
        print("Starting main function for setup and benchmark...")
        llm_engine, tokenizer, config = main()
        print("Main function execution finished.")

    except Exception as e:
        print(f"\n--- An critical error occurred during main execution ---")
        print(e)
        # import traceback
        # traceback.print_exc()

    # --- Start Interactive Mode AFTER main() if successful ---
    if llm_engine and tokenizer and config:
        print("\n\n--- Interactive Mode (vLLM) ---")
        print("Benchmark complete. Interactive mode starting.")
        print("Type 'exit' or press Ctrl+C to quit.")

        # Prepare sampling parameters for interactive use
        interactive_sampling_params = SamplingParams(
            n=1,              # Generate one completion
            temperature=0.7,  # Allow some creativity
            top_p=0.9,        # Nucleus sampling
            max_tokens=config.max_output_length # Use max output length from config
        )

        # Prepare LoRA request if LoRA was enabled during config setup
        # (Note: It's currently disabled by default in main for the max throughput run)
        interactive_lora_request = None
        if config.enable_lora and config.lora_modules:
             # Need to import LoRARequest if re-enabling LoRA
             # from vllm import LoRARequest # Uncomment if LoRA is enabled
             lora_info = config.lora_modules[0]
             interactive_lora_request = LoRARequest(
                lora_name=lora_info["lora_name"],
                lora_int_id=lora_info["lora_int_id"]
            )
             print(f"Interactive mode using LoRA: {interactive_lora_request.lora_name}")


        # The interactive loop
        while True:
            try:
                user_input = input("\nEnter prompt: ")
                if user_input.lower() == 'exit':
                    break
                if not user_input.strip(): # Handle empty input
                    continue

                # Generate response using the vLLM engine
                iter_start_time = time.time()
                vllm_outputs = llm_engine.generate(
                    prompts=[user_input], # Pass prompt as a list
                    sampling_params=interactive_sampling_params,
                    lora_request=interactive_lora_request # Pass None if LoRA disabled
                    )
                iter_end_time = time.time()

                # Extract and display results
                output = vllm_outputs[0] # Get the result for the first prompt
                response = output.outputs[0].text

                # Calculate performance metrics for this specific interaction
                input_tokens = len(output.prompt_token_ids)
                output_tokens = len(output.outputs[0].token_ids)
                total_tokens = input_tokens + output_tokens
                inference_time = iter_end_time - iter_start_time
                throughput = total_tokens / inference_time if inference_time > 0 else 0

                print("\nResponse:")
                print(response)
                print("\n--- Performance ---")
                print(f"Input tokens: {input_tokens}")
                print(f"Output tokens: {output_tokens}")
                print(f"Time: {inference_time:.2f} seconds")
                print(f"Throughput: {throughput:.2f} tokens/sec")

            except KeyboardInterrupt: # Allow graceful exit with Ctrl+C
                print("\nExiting interactive mode due to KeyboardInterrupt.")
                break
            except Exception as e:
                print(f"\nAn error occurred during interactive generation: {e}")
                # Consider adding more robust error handling if needed
                time.sleep(1) # Prevent rapid error loops in case of persistent issues

        print("\n--- Interactive Mode Finished ---")

    else:
        print("\nInteractive mode skipped due to error during main execution or missing objects.")

    # Final cleanup (optional, good practice)
    print("\n--- Script Execution Complete ---")
    print("Attempting final cleanup...")
    del llm_engine # Allow GC to collect the engine
    del tokenizer
    del config
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("Cleanup attempted.")

Starting main function for setup and benchmark...
--- Configuration for Max Throughput (Quantized) ---
Model ID: TheBloke/Mistral-7B-Instruct-v0.1-AWQ
Quantization: awq
LoRA Disabled
Max Model Length Override: 512
----------------------------------------------------

Loading tokenizer for TheBloke/Mistral-7B-Instruct-v0.1-AWQ...
Tokenizer loaded.
Padding side: left
Pad token: '</s>', ID: 2

Initializing vLLM engine with AWQ model...
WARNING 04-07 20:41:31 [config.py:2704] Casting torch.bfloat16 to torch.float16.
INFO 04-07 20:41:31 [config.py:600] This model supports multiple tasks: {'classify', 'score', 'reward', 'generate', 'embed'}. Defaulting to 'generate'.
WARNING 04-07 20:41:31 [config.py:679] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-07 20:41:31 [llm_engine.py:242] Initializing a V0 LLM engine (v0.8.3) with config: model='TheBloke/Mistral-7B-Instruct-v0.1-AWQ', speculative_config=None, tokenizer='TheBloke/Mistral-7B-I

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-07 20:41:44 [loader.py:447] Loading weights took 11.57 seconds
INFO 04-07 20:41:44 [model_runner.py:1146] Model loading took 3.8735 GiB and 12.183108 seconds
INFO 04-07 20:41:47 [worker.py:267] Memory profiling takes 1.72 seconds
INFO 04-07 20:41:47 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 04-07 20:41:47 [worker.py:267] model weights take 3.87GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 0.38GiB; the rest of the memory reserved for KV Cache is 9.02GiB.
INFO 04-07 20:41:47 [executor_base.py:112] # cuda blocks: 4617, # CPU blocks: 2048
INFO 04-07 20:41:47 [executor_base.py:117] Maximum concurrency for 512 tokens per request: 144.28x
INFO 04-07 20:41:48 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:47<00:00,  1.36s/it]

INFO 04-07 20:42:36 [model_runner.py:1598] Graph capturing finished in 48 secs, took 0.31 GiB
INFO 04-07 20:42:36 [llm_engine.py:448] init engine (profile, create kv cache, warmup model) took 51.15 seconds


...


Response:
 west is getting a lot of hate from the media and the public, it seems like everytime he opens his mouth people just want to shut him down and call him a racist. i think kanye is just trying to be a normal person and do what he loves, but people just want to judge him and label him as a racist.

i think people just want to shut kanye down because he is a celebrity and people think that celebrities are not allowed to have opinions or speak their minds. but i think that is ridiculous, everyone should be able to express themselves and have their own opinions.



--- Performance ---
Input tokens: 7
Output tokens: 128
Time: 4.44 seconds
Throughput: 30.40 tokens/sec

Enter prompt: kanye is dumb as fuck


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.44s/it, est. speed input: 1.80 toks/s, output: 28.83 toks/s]



Response:
. he has a mental disorder. he's not a musician. he's a puppet. he's a poser. he's a troll. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he's a poser. he'

--- Performance ---
Input tokens: 8
Output tokens: 128
Time: 4.45 seconds
Throughput: 30.59 tokens/sec

Enter prompt: nsfw roleplau


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.42s/it, est. speed input: 1.58 toks/s, output: 28.98 toks/s]



Response:


This is a nsfw roleplay.

Please be aware that the content within this roleplay may be mature or offensive.

If you are not comfortable with such content, please proceed with caution.

If you are under the age of 18, please do not enter this roleplay.

If you enter this roleplay, you agree to the following terms:

* I am of legal age to enter this roleplay.
* I am comfortable with mature or offensive content.
* I understand that this roleplay is nsfw and may not be suitable for all audiences.

--- Performance ---
Input tokens: 7
Output tokens: 128
Time: 4.42 seconds
Throughput: 30.54 tokens/sec

Enter prompt: exit

--- Interactive Mode Finished ---

--- Script Execution Complete ---
Attempting final cleanup...
Cleanup attempted.
